## Pascal VOC to tile mapping

ArcGIS Pro includes the ‘Export Training Data for Deep Learning’ tool that can be used to create labelled image chips that are needed to train a deep learning model. For object detection workflows, the output data format is in the "Pascal VOC (Visual Object Challenge)" format.

This notebooks reads in the labeled Pascal VOC format files, and creates a Python dictionary containing a mapping of each image chip to the center coordinates of each object within that file. This Python dictionary is then pickled and serves as an input in the Training notebook.

Another option to create image chips and the tile mapping is using ArcGIS API for Python, which has methods for exporting images from Imagery (eg NAIP imagery layers) as well as Tile layers (such as the Esri World Imagery layer). The 'Export Training Samples' notebook uses that approach. Only one of these notebooks will typically be used in the complete workflow.

In [1]:
from pathlib import Path

In [2]:
root = Path(r'Pools/FalseColor/')
labels = root / 'labels'

In [ ]:
import xml.etree.ElementTree as ET

tilemapping = {}

for xmlfile in labels.glob('*.xml'):
    print(xmlfile)
    jpgfile = xmlfile.name.replace('.xml', '.jpg')
    tree = ET.parse(xmlfile)
    xmlroot = tree.getroot()
    for child in xmlroot:
        centers = []
        if child.tag == 'object':
            xmin, ymin, xmax, ymax = float(child[1][0].text), float(child[1][1].text), float(child[1][2].text), float(child[1][3].text)
            x, y = (xmin + xmax) / 2.0, (ymin + ymax) / 2.0
            x, y = x/ 224.0, y/ 224.0
            centers.append((x, y))
        tilemapping[jpgfile] = centers

In [29]:
tilemapping['000000003.jpg']

[(0.1064732142857143, 0.0459375)]

In [32]:
import pickle

pickle.dump(tilemapping, open(r'Pools/FalseColor/tilemapping.pkl', 'wb'))